In [ ]:
import heapq
import time
import numpy as np
class GreedyRobotCargo:
    def __init__(self, grid_size, robot_positions, cargo_positions, cargo_destinations, time_steps):
            self.grid_size = grid_size
            self.initial_robot_positions = robot_positions
            self.initial_cargo_positions = cargo_positions
            self.cargo_destinations = cargo_destinations
            self.time_steps = time_steps
            self.blocked_positions = set()  # 一般的阻擋位置
            self.locked_positions = {}   # 貨物鎖死後的阻擋位置
            self.robot_paths = []
            self.available_cargoes = set(range(len(cargo_positions)))
            self.robot_status = [0] * len(robot_positions)  # 跟踪每個機器人的任務完成時間
            self.robot_tasks = {}  # 儲存每個機器人正在處理的貨物
            self.cargo_positions = cargo_positions[:]  # 複製以允許修改

    def block_path(self, path, cargo_id=None):
        for i in range(len(path) - 1):
            x1, y1, t1 = path[i]
            x2, y2, t2 = path[i + 1]


            self.blocked_positions.add((x1, y1, t1))
            self.blocked_positions.add((x2, y2, t2))
            
            # 封鎖雙向路徑，無論有無貨物
            self.blocked_positions.add(((x1, y1), (x2, y2), t1))
            self.blocked_positions.add(((x2, y2), (x1, y1), t1))
            if (x1, y1, t1) == (x2, y2, t2):
                # 上 (y1-1)
                self.blocked_positions.add(((x1, y1), (x1, y1-1), t1))
                self.blocked_positions.add(((x1, y1-1), (x1, y1), t1))

                # 下 (y1+1)
                self.blocked_positions.add(((x1, y1), (x1, y1+1), t1))
                self.blocked_positions.add(((x1, y1+1), (x1, y1), t1))

                # 左 (x1-1)
                self.blocked_positions.add(((x1, y1), (x1-1, y1), t1))
                self.blocked_positions.add(((x1-1, y1), (x1, y1), t1))

                # 右 (x1+1)
                self.blocked_positions.add(((x1, y1), (x1+1, y1), t1))
                self.blocked_positions.add(((x1+1, y1), (x1, y1), t1))


    def is_valid(self, x, y, t, prev_x=None, prev_y=None):
        # 確認座標是否在網格內
        if not (0 <= x < self.grid_size and 0 <= y < self.grid_size):
            return False

        # 檢查兩點之間的路徑是否被封鎖

        if prev_x is not None and prev_y is not None:
            if (((prev_x, prev_y), (x, y), t - 1) in self.blocked_positions or 
                ((x, y), (prev_x, prev_y), t - 1) in self.blocked_positions):
                return False
        if prev_x is not None and prev_y is not None:
            if (((prev_x, prev_y), (x, y), t) in self.blocked_positions or 
                ((x, y), (prev_x, prev_y), t) in self.blocked_positions):
                return False
        if prev_x is not None and prev_y is not None:
            if (((prev_x, prev_y), (x, y), t+1) in self.blocked_positions or 
                ((x, y), (prev_x, prev_y), t+1) in self.blocked_positions):
                return False


        # 檢查該點是否在時間t時被阻擋
        if (x, y, t) in self.blocked_positions:
            return False

        # 檢查該位置是否為已鎖定的貨物目的地
        if (x, y,t) in self.locked_positions:
            return False
        
        for robot_idx, cargo_idx in self.robot_tasks.items():
            cargo_pos = self.cargo_positions[cargo_idx]
            # 檢查該貨物是否在機器人運輸路徑上
            if cargo_pos == (x, y) and self.robot_status[robot_idx] > t:
                return False  # 如果有貨物在這裡且它正在被搬運，則這個位置是障礙物


        # 確認該時間點是否有其他機器人在此位置
        for robot_idx, path in self.robot_paths:
            for (rx, ry, rt) in path:
                if rt == t and (rx, ry) == (x, y):
                    return False  # 該位置被其他機器人佔據

        return True
    def neighbors(self, x, y, t):
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nx, ny = x + dx, y + dy
            if self.is_valid(nx, ny, t + 1, x, y):
                yield nx, ny, t + 1

    def shortest_path(self, start, goal, start_time=0):
        queue = [(0, start[0], start[1], start_time, [])]  # (cost, x, y, time, path)
        visited = set()
        while queue:
            cost, x, y, t, path = heapq.heappop(queue)
            if (x, y, t) in visited:
                continue
            visited.add((x, y, t))
            path = path + [(x, y, t)]
            if (x, y) == goal:
                return cost, path
            for nx, ny, nt in self.neighbors(x, y, t):
                heapq.heappush(queue, (cost + 1, nx, ny, nt, path))
        return float('inf'), []
    
    def is_locked(self, x, y, current_time):
        """檢查位置是否已經鎖定，僅當當前時間超過鎖定時間時才真正鎖定"""
        if (x, y) in self.locked_positions:
            lock_time = self.locked_positions[(x, y)]
            if current_time >= lock_time:
                print(f"位置 ({x}, {y}) 已鎖定，當前時間: {current_time}")
                return True
            else:
                print(f"位置 ({x}, {y}) 尚未鎖定，鎖定時間為 {lock_time}，當前時間: {current_time}")
                return False
        return False  # 如果位置不在鎖定列表中，則視為未鎖定

    def lock_position(self, x, y, lock_time):
        """在指定時間後鎖定已運送完成的貨物位置，防止其他機器人干擾"""
        self.locked_positions[(x, y)] = lock_time  # 位置和鎖定時間
        print(f"位置 ({x}, {y}) 將在時間 {lock_time} 之後鎖定，無法再搬動此貨物。")

    def find_nearest_empty_position(self, start_pos):
        min_dist = float('inf')
        nearest_pos = None
        for x in range(self.grid_size):
            for y in range(self.grid_size):
                pos = (x, y)
                if pos not in self.cargo_positions and (x, y) not in self.blocked_positions:
                    dist, _ = self.shortest_path(start_pos, pos)
                    if dist < min_dist:
                        min_dist = dist
                        nearest_pos = pos
        return nearest_pos

    def move_blocking_cargo(self, robot_idx, start_pos, cargo_id):
        # 獲取目標貨物的目的地
        target_cargo_pos = self.cargo_positions[cargo_id]
        target_cargo_dest = self.cargo_destinations[cargo_id]

        # 獲取到達目的地的路徑
        current_time = self.robot_status[robot_idx]
        
        path_to_target = self.shortest_path(start_pos, target_cargo_pos,current_time)[1]
        total_path = [] 
        path_to_dest = self.shortest_path(target_cargo_pos, target_cargo_dest,current_time)[1]
        total_path.extend([(x, y, t) for (x, y, t) in path_to_target])
        current_time += len(path_to_target)
        block = False        # 檢查路徑上的障礙物
        for x, y, t in path_to_dest:
            
            if (x, y) in self.cargo_positions and (x, y) != self.cargo_positions[cargo_id]:

                block = True
                blocking_cargo_idx = self.cargo_positions.index((x, y))  # 獲取阻擋的貨物索引
                path_to_block = self.shortest_path(target_cargo_pos, (x, y),current_time)[1]

                total_path.extend([(x, y, t) for (x, y, t) in path_to_block])
                current_time += len(path_to_block)
                # 尋找空位
                empty_pos = self.find_nearest_empty_position((x, y))
                if empty_pos:
                    # 計算從阻擋貨物位置到空位的路徑
                    path_to_empty = self.shortest_path((x, y), empty_pos,current_time)[1]
                    path_to_empty = path_to_empty[1:]
                    #self.block_path(path_to_empty)
                    total_path.extend([(x, y, t-1) for (x, y, t) in path_to_empty])
                    current_time += len(path_to_empty)
                    
                    print(f"  moving: {self.cargo_positions[blocking_cargo_idx]} to {empty_pos}")
                    print(total_path)
                    # 更新貨物位置
                    self.cargo_positions[blocking_cargo_idx] = empty_pos
                    self.update_cargo_positions()
                    
                    # 更新回到原來貨物位置的路徑
                    path_back_to_cargo = self.shortest_path(empty_pos,(x, y),current_time)[1]
                    path_back_to_cargo = path_back_to_cargo[1:]
                    #self.block_path(path_back_to_cargo)
                    print(total_path)
                    total_path.extend([(x, y, t-1) for (x, y, t) in path_back_to_cargo])
                    current_time += len(path_back_to_cargo)
                    place = (x,y) 
        if block and empty_pos:
            path_back_to_cargo = self.shortest_path(place,target_cargo_pos,current_time)[1]
            path_back_to_cargo = path_back_to_cargo[1:]
            #self.block_path(path_back_to_cargo)
            print(total_path)
            
            total_path.extend([(x, y, t-1) for (x, y, t) in path_back_to_cargo])
            current_time += len(path_back_to_cargo)
        else:
            print("無解")
        # 在最終路徑上進行搬移貨物的操作
        path_to_dest = self.shortest_path(target_cargo_pos, target_cargo_dest,current_time)[1]
        path_to_dest = path_to_dest[1:]
        total_path.extend([(x, y, t-1) for (x, y, t) in path_to_dest])
        current_time += len(path_to_dest)
        # 更新機器人的狀態

        self.robot_status[robot_idx] += len(total_path)

        self.block_path(total_path)
        print(f"  Total path: {total_path}")

        return total_path

    def allocate_tasks(self):
        start_times = time.time()

        times = 0
        while self.available_cargoes:
            all_robots_busy = True

            # 只考慮未完成的貨物
            self.available_cargoes = {cargo_idx for cargo_idx in self.available_cargoes 
                                      if self.cargo_destinations[cargo_idx] != (float('inf'), float('inf'))}

            for robot_idx in range(len(self.initial_robot_positions)):
                if times >= self.robot_status[robot_idx]:  # 檢查機器人是否可用
                    all_robots_busy = False
                    if self.available_cargoes:
                        min_dist = float('inf')
                        target_cargo = None
                        best_path_to_cargo = None

                        # 為機器人選擇最近的可用貨物
                        for cargo_idx in self.available_cargoes:
                            cargo_pos = self.cargo_positions[cargo_idx]
                            start_time = self.robot_status[robot_idx]  # 機器人的當前可用時間
                            dist, path_to_cargo = self.shortest_path(self.initial_robot_positions[robot_idx], cargo_pos, start_time)

                            # 打印檢查的信息
                            print(f"Robot {robot_idx + 1} checking cargo {cargo_idx + 1}:")
                            print(f"  Cargo position: {cargo_pos}")
                            print(f"  Path to cargo: {path_to_cargo}")
                            print(f"  Distance: {dist}")

                            if dist < min_dist:
                                min_dist = dist
                                target_cargo = cargo_idx
                                best_path_to_cargo = path_to_cargo

                        if target_cargo is not None:
                            cargo_dest = self.cargo_destinations[target_cargo]
                            cargo_pos = self.cargo_positions[target_cargo]
                            print(self.robot_paths)
                            blocking_path = self.move_blocking_cargo(robot_idx, self.initial_robot_positions[robot_idx], target_cargo)
                            
                            # 記錄機器人的路徑和貨物
                            robot_path =  blocking_path
                            
                            # 在到達目的地之前檢查是否需要等待
                            if self.path_passes_through(robot_path, robot_idx,cargo_pos,cargo_dest):
                                print(f"Robot {robot_idx + 1} is waiting for the target position to be free.")
                                continue  # 讓機器人等待，直到下一次迴圈再檢查
                           
                            self.robot_paths.append((f"R{robot_idx + 1}", robot_path))

                            # 更新貨物狀態
                            self.available_cargoes.remove(target_cargo)
                            robot_pos = robot_path[-1]
                            start_time_for_dest = robot_path[-1][2] + 1

                            self.initial_robot_positions[robot_idx] = cargo_dest
                            #self.block_path(path_to_dest)
                            # 鎖定貨物到達的目的地
                            final_pos = robot_path[-1][:2]  # 最後的目的地位置
                            self.cargo_positions[target_cargo]=final_pos
                            self.lock_position(cargo_dest[0], cargo_dest[1], robot_path[-1][2])
            times += 1
            if times >= self.time_steps:
                break

            self.update_cargo_positions()
            
            # 其餘代碼保持不變
            print(f"Available cargos: {self.available_cargoes}")
            print(f"Robot statuses: {self.robot_status}")
            print(f"Cargo positions: {self.cargo_positions}")
        
        end_times = time.time()
        execution_time = end_times - start_times
        print(f"總執行時間：{execution_time:.2f} 秒")
        print(self.robot_paths)

    def path_passes_through(self, path, current_robot_idx,cargo_pos,cargo_dest):
        """檢查路徑是否經過其他機器人的目標位置"""
        # 將可用貨物集合轉換為列表
        available_cargos_list = list(self.available_cargoes)


        # 檢查當前路徑的所有位置
        path_positions = set((x, y) for x, y, t in path)

        # 檢查是否有其他機器人的路徑會經過當前機器人的目標位置
        for robot_name, robot_path in self.robot_paths:
            if robot_name != f"R{current_robot_idx + 1}":  # 忽略當前機器人的路徑
                for (x, y, t) in robot_path:
                    if (x, y) == cargo_dest and t >= self.robot_status[current_robot_idx]-1:  # 確保時間大於當前時間
                        current_time = self.robot_status[current_robot_idx]
                        return True  # 檢測到經過其他機器人的目標位置
         
        

        return False  # 沒有經過


    def update_cargo_positions(self):
        new_positions = list(self.cargo_positions)

        for robot_idx, cargo_idx in self.robot_tasks.items():
            robot_pos = self.initial_robot_positions[robot_idx]

            if cargo_idx in self.cargo_positions:
                if self.cargo_positions[cargo_idx] == robot_pos:
                    new_positions[cargo_idx] = robot_pos 

        self.cargo_positions = new_positions
        print(f"Updated cargo positions: {self.cargo_positions}")

    def print_grids(self):
        max_time = max((t for _, path in self.robot_paths for (_, _, t) in path), default=0)
        for t in range(max_time + 1):
            grid = [["." for _ in range(self.grid_size)] for _ in range(self.grid_size)]
            for (robot_idx, path) in self.robot_paths:
                for (x, y, time) in path:
                    if time == t:
                        grid[y][x] = robot_idx
            print(f"Time {t}:")
            for row in grid:
                print(" ".join(row))
            print()



In [ ]:
import heapq
import time
import numpy as np
class GreedyRobotCargo:
    def __init__(self, grid_size, robot_positions, cargo_positions, cargo_destinations, time_steps):
            self.grid_size = grid_size
            self.initial_robot_positions = robot_positions
            self.initial_cargo_positions = cargo_positions
            self.cargo_destinations = cargo_destinations
            self.time_steps = time_steps
            self.blocked_positions = set()  # 一般的阻擋位置
            self.locked_positions = {}   # 貨物鎖死後的阻擋位置
            self.robot_paths = []
            self.available_cargoes = set(range(len(cargo_positions)))
            self.robot_status = [0] * len(robot_positions)  # 跟踪每個機器人的任務完成時間
            self.robot_tasks = {}  # 儲存每個機器人正在處理的貨物
            self.cargo_positions = cargo_positions[:]  # 複製以允許修改

    def block_path(self, path, cargo_id=None):
        for i in range(len(path) - 1):
            x1, y1, t1 = path[i]
            x2, y2, t2 = path[i + 1]


            self.blocked_positions.add((x1, y1, t1))
            self.blocked_positions.add((x2, y2, t2))
            
            # 封鎖雙向路徑，無論有無貨物
            self.blocked_positions.add(((x1, y1), (x2, y2), t1))
            self.blocked_positions.add(((x2, y2), (x1, y1), t1))
            if (x1, y1, t1) == (x2, y2, t2):
                # 上 (y1-1)
                self.blocked_positions.add(((x1, y1), (x1, y1-1), t1))
                self.blocked_positions.add(((x1, y1-1), (x1, y1), t1))

                # 下 (y1+1)
                self.blocked_positions.add(((x1, y1), (x1, y1+1), t1))
                self.blocked_positions.add(((x1, y1+1), (x1, y1), t1))

                # 左 (x1-1)
                self.blocked_positions.add(((x1, y1), (x1-1, y1), t1))
                self.blocked_positions.add(((x1-1, y1), (x1, y1), t1))

                # 右 (x1+1)
                self.blocked_positions.add(((x1, y1), (x1+1, y1), t1))
                self.blocked_positions.add(((x1+1, y1), (x1, y1), t1))


    def is_valid(self, x, y, t, prev_x=None, prev_y=None):
        # 確認座標是否在網格內
        if not (0 <= x < self.grid_size and 0 <= y < self.grid_size):
            return False

        # 檢查兩點之間的路徑是否被封鎖

        if prev_x is not None and prev_y is not None:
            if (((prev_x, prev_y), (x, y), t - 1) in self.blocked_positions or 
                ((x, y), (prev_x, prev_y), t - 1) in self.blocked_positions):
                return False
        if prev_x is not None and prev_y is not None:
            if (((prev_x, prev_y), (x, y), t) in self.blocked_positions or 
                ((x, y), (prev_x, prev_y), t) in self.blocked_positions):
                return False
        if prev_x is not None and prev_y is not None:
            if (((prev_x, prev_y), (x, y), t+1) in self.blocked_positions or 
                ((x, y), (prev_x, prev_y), t+1) in self.blocked_positions):
                return False


        # 檢查該點是否在時間t時被阻擋
        if (x, y, t) in self.blocked_positions:
            return False

        # 檢查該位置是否為已鎖定的貨物目的地
        if (x, y,t) in self.locked_positions:
            return False
        
        for robot_idx, cargo_idx in self.robot_tasks.items():
            cargo_pos = self.cargo_positions[cargo_idx]
            # 檢查該貨物是否在機器人運輸路徑上
            if cargo_pos == (x, y) and self.robot_status[robot_idx] > t:
                return False  # 如果有貨物在這裡且它正在被搬運，則這個位置是障礙物


        # 確認該時間點是否有其他機器人在此位置
        for robot_idx, path in self.robot_paths:
            for (rx, ry, rt) in path:
                if rt == t and (rx, ry) == (x, y):
                    return False  # 該位置被其他機器人佔據

        return True
    def neighbors(self, x, y, t):
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nx, ny = x + dx, y + dy
            if self.is_valid(nx, ny, t + 1, x, y):
                yield nx, ny, t + 1

    def shortest_path(self, start, goal, start_time=0):
        queue = [(0, start[0], start[1], start_time, [])]  # (cost, x, y, time, path)
        visited = set()
        while queue:
            cost, x, y, t, path = heapq.heappop(queue)
            if (x, y, t) in visited:
                continue
            visited.add((x, y, t))
            path = path + [(x, y, t)]
            if (x, y) == goal:
                return cost, path
            for nx, ny, nt in self.neighbors(x, y, t):
                heapq.heappush(queue, (cost + 1, nx, ny, nt, path))
        return float('inf'), []
    
    def is_locked(self, x, y, current_time):
        """檢查位置是否已經鎖定，僅當當前時間超過鎖定時間時才真正鎖定"""
        if (x, y) in self.locked_positions:
            lock_time = self.locked_positions[(x, y)]
            if current_time >= lock_time:
      #          print(f"位置 ({x}, {y}) 已鎖定，當前時間: {current_time}")
                return True
            else:
     #           print(f"位置 ({x}, {y}) 尚未鎖定，鎖定時間為 {lock_time}，當前時間: {current_time}")
                return False
        return False  # 如果位置不在鎖定列表中，則視為未鎖定

    def lock_position(self, x, y, lock_time):
        """在指定時間後鎖定已運送完成的貨物位置，防止其他機器人干擾"""
        self.locked_positions[(x, y)] = lock_time  # 位置和鎖定時間
     #   print(f"位置 ({x}, {y}) 將在時間 {lock_time} 之後鎖定，無法再搬動此貨物。")

    def find_nearest_empty_position(self, start_pos):
        min_dist = float('inf')
        nearest_pos = None
        for x in range(self.grid_size):
            for y in range(self.grid_size):
                pos = (x, y)
                if pos not in self.cargo_positions and (x, y) not in self.blocked_positions:
                    dist, _ = self.shortest_path(start_pos, pos)
                    if dist < min_dist:
                        min_dist = dist
                        nearest_pos = pos
        return nearest_pos

    def move_blocking_cargo(self, robot_idx, start_pos, cargo_id):
        # 獲取目標貨物的目的地
        target_cargo_pos = self.cargo_positions[cargo_id]
        target_cargo_dest = self.cargo_destinations[cargo_id]

        # 獲取到達目的地的路徑
        current_time = self.robot_status[robot_idx]
        
        path_to_target = self.shortest_path(start_pos, target_cargo_pos,current_time)[1]
        total_path = [] 
        path_to_dest = self.shortest_path(target_cargo_pos, target_cargo_dest,current_time)[1]
        total_path.extend([(x, y, t) for (x, y, t) in path_to_target])
        current_time += len(path_to_target)
        block = False        # 檢查路徑上的障礙物
        for x, y, t in path_to_dest:
            
            if (x, y) in self.cargo_positions and (x, y) != self.cargo_positions[cargo_id]:

                block = True
                blocking_cargo_idx = self.cargo_positions.index((x, y))  # 獲取阻擋的貨物索引
                path_to_block = self.shortest_path(target_cargo_pos, (x, y),current_time)[1]

                total_path.extend([(x, y, t) for (x, y, t) in path_to_block])
                current_time += len(path_to_block)
                # 尋找空位
                empty_pos = self.find_nearest_empty_position((x, y))
                if empty_pos:
                    # 計算從阻擋貨物位置到空位的路徑
                    path_to_empty = self.shortest_path((x, y), empty_pos,current_time)[1]
                    path_to_empty = path_to_empty[1:]
                    #self.block_path(path_to_empty)
                    total_path.extend([(x, y, t-1) for (x, y, t) in path_to_empty])
                    current_time += len(path_to_empty)
                    
               #     print(f"  moving: {self.cargo_positions[blocking_cargo_idx]} to {empty_pos}")
                #    print(total_path)
                    # 更新貨物位置
                    self.cargo_positions[blocking_cargo_idx] = empty_pos

                    # 更新回到原來貨物位置的路徑
                    path_back_to_cargo = self.shortest_path(empty_pos,(x, y),current_time)[1]
                    path_back_to_cargo = path_back_to_cargo[1:]
                    #self.block_path(path_back_to_cargo)
              #      print(total_path)
                    total_path.extend([(x, y, t-1) for (x, y, t) in path_back_to_cargo])
                    current_time += len(path_back_to_cargo)
                    place = (x,y) 
        if block and empty_pos:
            path_back_to_cargo = self.shortest_path(place,target_cargo_pos,current_time)[1]
            path_back_to_cargo = path_back_to_cargo[1:]
            #self.block_path(path_back_to_cargo)
          #  print(total_path)
            
            total_path.extend([(x, y, t-1) for (x, y, t) in path_back_to_cargo])
            current_time += len(path_back_to_cargo)
        #在最終路徑上進行搬移貨物的操作
        path_to_dest = self.shortest_path(target_cargo_pos, target_cargo_dest,current_time)[1]
        path_to_dest = path_to_dest[1:]
        total_path.extend([(x, y, t-1) for (x, y, t) in path_to_dest])
        current_time += len(path_to_dest)
        # 更新機器人的狀態

        self.robot_status[robot_idx] += len(total_path)

        self.block_path(total_path)
        #print(f"  Total path: {total_path}")

        return total_path

    def allocate_tasks(self):
        start_times = time.time()

        times = 0
        while self.available_cargoes:
            all_robots_busy = True

            # 只考慮未完成的貨物
            self.available_cargoes = {cargo_idx for cargo_idx in self.available_cargoes 
                                      if self.cargo_destinations[cargo_idx] != (float('inf'), float('inf'))}

            for robot_idx in range(len(self.initial_robot_positions)):
                if times >= self.robot_status[robot_idx]:  # 檢查機器人是否可用
                    all_robots_busy = False
                    if self.available_cargoes:
                        min_dist = float('inf')
                        target_cargo = None
                        best_path_to_cargo = None

                        # 為機器人選擇最近的可用貨物
                        for cargo_idx in self.available_cargoes:
                            cargo_pos = self.cargo_positions[cargo_idx]
                            start_time = self.robot_status[robot_idx]  # 機器人的當前可用時間
                            dist, path_to_cargo = self.shortest_path(self.initial_robot_positions[robot_idx], cargo_pos, start_time)

                            # 打印檢查的信息
                           # print(f"Robot {robot_idx + 1} checking cargo {cargo_idx + 1}:")
                           # print(f"  Cargo position: {cargo_pos}")
                           # print(f"  Path to cargo: {path_to_cargo}")
                           # print(f"  Distance: {dist}")

                            if dist < min_dist:
                                min_dist = dist
                                target_cargo = cargo_idx
                                best_path_to_cargo = path_to_cargo

                        if target_cargo is not None:
                            cargo_dest = self.cargo_destinations[target_cargo]
                            cargo_pos = self.cargo_positions[target_cargo]
                            #print(self.robot_paths)
                            blocking_path = self.move_blocking_cargo(robot_idx, self.initial_robot_positions[robot_idx], target_cargo)
                            
                            # 記錄機器人的路徑和貨物
                            robot_path =  blocking_path
                            
                            # 在到達目的地之前檢查是否需要等待
                            if self.path_passes_through(robot_path, robot_idx,cargo_pos,cargo_dest):
                                #print(f"Robot {robot_idx + 1} is waiting for the target position to be free.")
                                continue  # 讓機器人等待，直到下一次迴圈再檢查
                           
                            self.robot_paths.append((f"R{robot_idx + 1}", robot_path))

                            # 更新貨物狀態
                            self.available_cargoes.remove(target_cargo)
                            robot_pos = robot_path[-1]
                            start_time_for_dest = robot_path[-1][2] + 1

                            self.initial_robot_positions[robot_idx] = cargo_dest
                            #self.block_path(path_to_dest)
                            # 鎖定貨物到達的目的地
                            final_pos = robot_path[-1][:2]  # 最後的目的地位置
                            self.cargo_positions[target_cargo]=final_pos
                            self.lock_position(cargo_dest[0], cargo_dest[1], robot_path[-1][2])
            times += 1
            if times >= self.time_steps:
                break


       # print("Robot paths:", self.robot_paths)
       # print("Paths extracted from robot paths:", [p[1] for p in self.robot_paths])
        end_time = [path[-1][2] for _, path in self.robot_paths]
        
        self.longest_time = max(end_time, default=0)
        print(self.longest_time,self.robot_paths,end_time)
        end_times = time.time()
        execution_time = end_times - start_times
        return {"runtime": f"{execution_time:.2f}", "obj": self.longest_time}
        

    def path_passes_through(self, path, current_robot_idx,cargo_pos,cargo_dest):
        """檢查路徑是否經過其他機器人的目標位置"""
        # 將可用貨物集合轉換為列表
        available_cargos_list = list(self.available_cargoes)


        # 檢查當前路徑的所有位置
        path_positions = set((x, y) for x, y, t in path)

        # 檢查是否有其他機器人的路徑會經過當前機器人的目標位置
        for robot_name, robot_path in self.robot_paths:
            if robot_name != f"R{current_robot_idx + 1}":  # 忽略當前機器人的路徑
                for (x, y, t) in robot_path:
                    if (x, y) == cargo_dest and t >= self.robot_status[current_robot_idx]-1:  # 確保時間大於當前時間
                        current_time = self.robot_status[current_robot_idx]
                        return True  # 檢測到經過其他機器人的目標位置
         
        

        return False  # 沒有經過

    def print_grids(self):
        max_time = max((t for _, path in self.robot_paths for (_, _, t) in path), default=0)
        for t in range(max_time + 1):
            grid = [["." for _ in range(self.grid_size)] for _ in range(self.grid_size)]
            for (robot_idx, path) in self.robot_paths:
                for (y,x, time) in path:
                    if time == t:
                        grid[y][x] = robot_idx
            print(f"Time {t}:")
            for row in grid:
                print(" ".join(row))
            print()
  


In [ ]:
import os
import pandas as pd
import re
import time

def parse_files(start_file, dest_file):
    with open(start_file, 'r') as start, open(dest_file, 'r') as dest:
        start_lines = start.readlines()
        dest_lines = dest.readlines()
    
    num_robots = 0
    robot_positions = []

    start_grid = [list(map(int, line.strip().split(','))) for line in start_lines[1 + num_robots:]]
    dest_grid = [list(map(int, line.strip().split(','))) for line in dest_lines[1 + num_robots:]]

    cargo_positions = {}
    cargo_destinations = {}
    obstacles = []

    for y, row in enumerate(start_grid):
        for x, cell in enumerate(row):
            if cell > 0:  # 貨物
                cargo_positions[cell] = (y, x)
            elif cell == -1:  # 障礙物
                obstacles.append((y, x))

    for y, row in enumerate(dest_grid):
        for x, cell in enumerate(row):
            if cell > 0:
                cargo_destinations[cell] = (y, x)

    sorted_cargo_positions = [cargo_positions[i] for i in sorted(cargo_positions)]
    sorted_cargo_destinations = [cargo_destinations[i] for i in sorted(cargo_destinations)]
    
    robot_positions = sorted_cargo_positions
    sorted_cargo_positions.extend(obstacles)
    sorted_cargo_destinations.extend([(float('inf'), float('inf'))] * len(obstacles))

    grid_size = len(start_grid)
    print(sorted_cargo_destinations)
    return grid_size, robot_positions, sorted_cargo_positions, sorted_cargo_destinations

def process_all_data(base_dir):
    results = []
    start_folder = os.path.join(base_dir, "start")
    end_folder = os.path.join(base_dir, "end")
    
    if not (os.path.exists(start_folder) and os.path.exists(end_folder)):
        return results

    for start_file in os.listdir(start_folder):
        if not start_file.endswith('.txt') or not start_file.startswith('graph_start_'):
            continue

        corresponding_end_file = start_file.replace("graph_start_", "graph_end_")
        start_file_path = os.path.join(start_folder, start_file)
        end_file_path = os.path.join(end_folder, corresponding_end_file)

        if not os.path.exists(end_file_path):
            print(f"End file not found for: {start_file}")
            continue
        print(start_file_path)
        grid_size, robot_positions, cargo_positions, cargo_destinations = parse_files(start_file_path, end_file_path)

        time_steps = 100
        allocator = GreedyRobotCargo(grid_size, robot_positions, cargo_positions, cargo_destinations, time_steps)
        result = allocator.allocate_tasks()

        file_name = os.path.basename(start_file)
        match = re.match(r"graph_start_(\d+x\d+)_tar(\d+)_sp(\d+)_(\d+).txt", file_name)

        if match:
            grid_size_str, tar_num, sp_num, test_num = match.groups()
            results.append({
                "n_tar": int(tar_num),
                "n_spa": int(sp_num),
                "id": int(test_num),
                "runtime": result["runtime"],
                "obj": result["obj"],
            })

    return results

def main():
    base_dir = r"C:\\Users\\mirac\\5x5"
    output_excel = r"C:\\Users\\mirac\\5x5.xlsx"
    tar_folders = ["tar1","tar2", "tar3", "tar4", "tar5","tar6"]

    all_results = []

    for tar in tar_folders:
        folder_path = os.path.join(base_dir, tar)
        print(f"Processing folder: {folder_path}")
        results = process_all_data(folder_path)
        if results:
            all_results.extend(results)

    if all_results:
        df = pd.DataFrame(all_results)
        df.to_excel(output_excel, index=False)
        print(f"Results saved to {output_excel}")
    else:
        print("No results to save.")

if __name__ == "__main__":
    main()

